# Day 26: 物体検出入門

## Learning Objectives
- 物体検出の概念を理解する
- 2段階検出（Two-stage Detection）を理解する
- 一段階検出（One-stage Detection）を理解する
- YOLOの基本原理を理解する

---

# Part 1: Theory (2 hours)

## 1.1 物体検出とは？

物体検出（Object Detection）は、画像内に存在する物体を検出し、その位置とクラスを特定するタスクです。物体検出は以下の情報を出力します：

1. **Bounding Box（バウンディングボックス）**: 物体を囲む矩形
2. **Class Label（クラスラベル）**: 物体の種類（人、車、猫など）
3. **Confidence Score（信頼度スコア）**: 検出の確信度

**物体検出の応用**:
- 自動運転
- 監視カメラ
- 医療診断
- 動物検出
- 欠陥検出

**画像分類 vs 物体検出**:
- 画像分類：画像全体が何かを分類
- 物体検出：画像内の複数の物体を個別に検出

## 1.2 物体検出のアーキテクチャ

物体検出のアーキテクチャは大きく2つのカテゴリに分けられます：

### 1. 2段階検出（Two-stage Detection）

**特徴**:
- 高精度
- 計算コストが高い
- R-CNN系列（R-CNN, Fast R-CNN, Faster R-CNN）

**処理フロー**:
1. Region Proposal（領域提案）- 候補領域を生成
2. Classification & Bounding Box Regression - 各領域を分類と回帰

**代表的なモデル**:
- Faster R-CNN
- Mask R-CNN（セグメンテーションも追加）
### 2. 一段階検出（One-stage Detection）

**特徴**:
- 高速
- 精度はやや低い
- YOLO, SSDなど

**処理フロー**:
1. グリッドを画像に分割
2. 各グリッドセルが物体を検出

**代表的なモデル**:
- YOLO (You Only Look Once)
- SSD (Single Shot MultiBox Detector)
- RetinaNet

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import cv2
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

print(f"TensorFlow version: {tf.__version__}")

## 1.3 YOLOの基本原理

YOLO（You Only Look Once）は最も人気のある一段階検出アルゴリズムです。

### YOLOの特徴

**特徴**:
- 画像を一度だけ見て（Look Once）検出を実行
- エンドツーエンドで学習可能
- 実時間処理に適している

**基本アイデア**:
1. 画像をS×Sのグリッドに分割
2. 各グリッドセルはB個のバウンディングボックスを予測
3. 各ボックスには以下を予測：
   - 中心座標 (x, y)
   - 幅と高さ (w, h)
   - 信頼度スコア
   - クラス確率分布

**出力形式**:
   - 各グリッドセルは(B×5 + C)個の出力を持つ
   - 5 = (x, y, w, h, confidence)
   - C = クラス数

In [ ]:
def create_yolo_grid_demo(S=7, B=2, C=20):
    """YOLOグリッドのデモンストレーション"""
    # グリッドの作成
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 7))
    
    # 左：グリッドの表示
    ax1.set_xlim(0, S)
    ax1.set_ylim(0, S)
    ax1.set_aspect('equal')
    ax1.grid(True, alpha=0.3)
    ax1.set_title(f'S×S = {S}×{S} グリッド')
    ax1.set_xlabel('X')
    ax1.set_ylabel('Y')
    
    # グリッドの境界線を描画
    for i in range(S + 1):
        ax1.axhline(y=i, color='black', linewidth=1)
        ax1.axvline(x=i, color='black', linewidth=1)
    
    # グリッドセル番号を表示
    for i in range(S):
        for j in range(S):
            cell_id = i * S + j
            ax1.text(j + 0.5, S - i - 0.5, f'{cell_id}', 
                    ha='center', va='center', fontsize=8)
    
    # 右：各セルの出力の説明
    ax2.axis('off')
    ax2.set_title('各グリッドセルの出力形式')
    
    # 出力形式の説明
    output_text = f"各セルの出力形式:\n\n"
    output_text += f"- バウンディングボックス数: {B}\n"
    output_text += f"- 各ボックスのパラメータ: 5個\n"
    output_text += f"  (x, y, w, h, confidence)\n"
    output_text += f"- クラス数: {C}\n\n"
    output_text += f"総出力サイズ = {S} × {S} × ({B} × 5 + {C})\n\n"
    output_text += f"= {S*S} × ({B*5 + C)} = {S*S*(B*5 + C)} 個の予測値\n\n"
    output_text += f"例: S=7, B=2, C=20:\n"
    output_text += f"= 7×7×(2×5 + 20) = 49 × 30 = 1470 個の予測値"
    
    ax2.text(0.1, 0.5, output_text, transform=ax2.transAxes,
             fontsize=12, verticalalignment='center',
             bbox=dict(boxstyle="round,pad=0.3", facecolor="lightgray"))
    
    plt.tight_layout()
    plt.show()

# YOLOグリッドデモの表示
create_yolo_grid_demo()

## 1.4 バウンディングボックスの予測

YOLOは相対座標でバウンディングボックスを予測します。

In [ ]:
def visualize_yolo_bbox_prediction(S=7, img_size=448):
    """YOLOのバウンディングボックス予測を可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # グリッドサイズ
    cell_size = img_size // S
    
    # 1. グリッドとセルの中心
    ax = axes[0, 0]
    ax.set_xlim(0, img_size)
    ax.set_ylim(0, img_size)
    ax.set_aspect('equal')
    ax.set_title('グリッドとセル中心')
    
    # グリッド描画
    for i in range(S + 1):
        ax.axhline(y=i * cell_size, color='gray', linewidth=1)
        ax.axvline(x=i * cell_size, color='gray', linewidth=1)
    
    # セル中心を表示
    centers = []
    for i in range(S):
        for j in range(S):
            cx = j * cell_size + cell_size // 2
            cy = i * cell_size + cell_size // 2
            ax.plot(cx, cy, 'bo', markersize=5)
            centers.append((cx, cy))
    
    # 2. 物体のあるセル
    ax = axes[0, 1]
    ax.set_xlim(0, img_size)
    ax.set_ylim(0, img_size)
    ax.set_aspect('equal')
    ax.set_title('物体のあるセル（セル相対座標）')
    
    # グリッド描画
    for i in range(S + 1):
        ax.axhline(y=i * cell_size, color='gray', linewidth=1)
        ax.axvline(x=i * cell_size, color='gray', linewidth=1)
    
    # 物体のあるセル（例: 中心に一つ）
    obj_cell_i, obj_cell_j = 3, 2  # 中央付近のセル
    obj_cx = obj_cell_j * cell_size + cell_size // 2
    obj_cy = obj_cell_i * cell_size + cell_size // 2
    
    # 物体の真の中心（グリッド相対）
    true_x_in_cell = 0.7  # セル内でのx位置（0-1）
    true_y_in_cell = 0.3  # セル内でのy位置（0-1）
    true_w_in_cell = 0.6  # セルに対する幅の割合
    true_h_in_cell = 0.8  # セルに対する高さの割合
    
    # セル相対座標でボックスを描画
    bbox_x = true_x_in_cell * cell_size
    bbox_y = true_y_in_cell * cell_size
    bbox_w = true_w_in_cell * cell_size
    bbox_h = true_h_in_cell * cell_size
    
    rect = plt.Rectangle((bbox_x, bbox_y), bbox_w, bbox_h,
                         linewidth=2, edgecolor='red', facecolor='none')
    ax.add_patch(rect)
    
    # セル中心を表示
    ax.plot(obj_cx, obj_cy, 'go', markersize=8)
    
    # ラベル
    ax.text(10, img_size-20, f"セル相対座標: ({true_x_in_cell:.2f}, {true_y_in_cell:.2f}, {true_w_in_cell:.2f}, {true_h_in_cell:.2f})",
            fontsize=10, bbox=dict(boxstyle="round,pad=0.3", facecolor="yellow", alpha=0.7))
    
    # 3. 絶対座標でのバウンディングボックス
    ax = axes[1, 0]
    ax.set_xlim(0, img_size)
    ax.set_ylim(0, img_size)
    ax.set_aspect('equal')
    ax.set_title('絶対座標でのバウンディングボックス')
    
    # グリッド描画
    for i in range(S + 1):
        ax.axhline(y=i * cell_size, color='gray', linewidth=1)
        ax.axvline(x=i * cell_size, color='gray', linewidth=1)
    
    # 絶対座標を計算
    abs_x = obj_cx + bbox_x - cell_size // 2
    abs_y = obj_cy + bbox_y - cell_size // 2
    abs_w = bbox_w
    abs_h = bbox_h
    
    # 絶対座標のボックスを描画
    rect_abs = plt.Rectangle((abs_x, abs_y), abs_w, abs_h,
                            linewidth=3, edgecolor='blue', facecolor='none')
    ax.add_patch(rect_abs)
    
    # 中心点を表示
    center_x = abs_x + abs_w / 2
    center_y = abs_y + abs_h / 2
    ax.plot(center_x, center_y, 'ro', markersize=8)
    
    # 4. 変換式
    ax = axes[1, 1]
    ax.axis('off')
    ax.set_title('YOLOの変換式')
    
    formula_text = """YOLOの座標変換:

# 相対座標 → 絶対座標
t_x = cell_x / S
t_y = cell_y / S
t_w = log(w / cell_w) / S
t_h = log(h / cell_h) / S

# 絶対座標 → 相対座標
cell_x = S * t_x
cell_y = S * t_y
w = exp(t_w * S) * cell_w
h = exp(t_h * S) * cell_h

# セル中心を考慮した最終的なバウンディングボックス
bx = S * t_x + cell_j
by = S * t_y + cell_i
bw = exp(t_w * S)
bh = exp(t_h * S)
"""
    
    ax.text(0.1, 0.5, formula_text, transform=ax.transAxes,
            fontsize=11, verticalalignment='center',
            fontfamily='monospace',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="lightblue"))
    
    plt.tight_layout()
    plt.show()

# YOLOのバウンディングボックス予測を可視化
visualize_yolo_bbox_prediction()

## 1.5 損失関数

YOLOの損失関数は3つの部分から構成されます：

In [ ]:
def plot_yolo_loss_components():
    """YOLOの損失関数の構成要素を可視化"""
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. 位置損失
    ax = axes[0, 0]
    x = np.linspace(0, 1, 100)
    y1 = x**2  # (x_pred - x_true)^2
    y2 = np.sqrt(x)  # sqrt(|x_pred - x_true|)
    
    ax.plot(x, y1, 'b-', label='x^2', linewidth=2)
    ax.plot(x, y2, 'r--', label='sqrt(|x|)', linewidth=2)
    ax.set_xlabel('誤差の大きさ')
    ax.set_ylabel('損失')
    ax.set_title('位置損失関数')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 2. 予測誤差と重み
    ax = axes[0, 1]
    errors = np.linspace(0, 1, 100)
    no_weight = errors**2
    with_weight = 5 * errors**2  # 予測がないセルの重み
    
    ax.plot(errors, no_weight, 'b-', label='通常のセル', linewidth=2)
    ax.plot(errors, with_weight, 'r--', label='予測がないセル', linewidth=2)
    ax.set_xlabel('誤差の大きさ')
    ax.set_ylabel('損失')
    ax.set_title('重み付き損失')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    # 3. クラス損失
    ax = axes[1, 0]
    # 交差エントロピー
    true_probs = [0.9, 0.05, 0.05]  # 真のクラス分布
    pred_probs_1 = [0.9, 0.05, 0.05]  # 完全に正解
    pred_probs_2 = [0.1, 0.45, 0.45]  # 異なるクラスに予測
    
    # 交差エントロピー計算
    def cross_entropy(true, pred):
        return -sum(t * np.log(p + 1e-10) for t, p in zip(true, pred))
    
    ce_1 = cross_entropy(true_probs, pred_probs_1)
    ce_2 = cross_entropy(true_probs, pred_probs_2)
    
    # バーの描画
    bars = ax.bar(['正確な予測', "不正確な予測"], [ce_1, ce_2], 
                 color=['green', 'red'])
    ax.set_ylabel('交差エントロピー損失')
    ax.set_title('クラス損失の例')
    ax.set_ylim(0, 2.5)
    
    # 数値を表示
    for bar, value in zip(bars, [ce_1, ce_2]):
        height = bar.get_height()
        ax.text(bar.get_x() + bar.get_width()/2., height,
                f'{value:.3f}', ha='center', va='bottom')
    
    # 4. 全体の損失構成
    ax = axes[1, 1]
    ax.axis('off')
    ax.set_title('YOLOの総損失関数')
    
    loss_text = """総損失 = λ_coord × L_coord + λ_noobj × L_noobj + L_class

1. 位置損失 L_coord:
   - 中心座標の誤差 (x, y)
   - 大きさの誤差 (w, h)
   - 重み λ_coord = 5

2. 予測なし損失 L_noobj:
   - 物体のないセルでの予測誤差
   - 重み λ_noobj = 0.5

3. クラス損失 L_class:
   - クラス予測の交差エントロピー
   - 全セルで計算

特徴:
- 位置と大きさには大きな重みを設定
- 物体のないセルでの予測をペナルティ
- クラス分離に大きく影響"""
    
    ax.text(0.05, 0.95, loss_text, transform=ax.transAxes,
            fontsize=11, verticalalignment='top',
            bbox=dict(boxstyle="round,pad=0.5", facecolor="lightyellow"))
    
    plt.tight_layout()
    plt.show()

# YOLOの損失関数を可視化
plot_yolo_loss_components()

---

# Part 2: Practice (2 hours)

それでは、学んだ知識を実際に使ってみましょう！

## Exercise 26.1: YOLOの基本的な実装

簡易版YOLOを実装し、バウンディングボックスの予測を行いましょう。

In [ ]:
def create_simple_yolo_detector(grid_size=7, num_boxes=2, num_classes=20):
    """簡易YOLO検出器の作成"""
    # モデルの入力
    inputs = keras.Input(shape=(448, 448, 3))
    
    # 特徴抽出（簡略化）
    x = layers.Conv2D(64, 7, strides=2, padding='same', activation='relu')(inputs)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(192, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(384, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    
    x = layers.Conv2D(512, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    
    # 最終的な特徴マップ
    feature_map = layers.Conv2D(1024, 3, padding='same', activation='relu')(x)
    
    # YOLO出力層
    output = layers.Conv2D(
        grid_size * grid_size * (num_boxes * 5 + num_classes),
        1,
        activation='sigmoid'
    )(feature_map)
    
    # 出力をreshape
    output = layers.Reshape((
        grid_size, grid_size, num_boxes, 5 + num_classes
    ))(output)
    
    # モデルの作成
    model = keras.Model(inputs=inputs, outputs=output)
    
    return model

# 簡易YOLOモデルの作成
model = create_simple_yolo_detector(grid_size=7, num_boxes=2, num_classes=20)
model.summary()

# モデルの図
keras.utils.plot_model(
    model, 
    show_shapes=True,
    show_layer_names=True,
    rankdir='TB',
    dpi=96
)

In [ ]:
def yolo_decode_predictions(predictions, grid_size=7, num_boxes=2, num_classes=20):
    """YOLOの出力をデコードする"""
    # 予測を取得
    batch_size = predictions.shape[0]
    
    # 各予測を展開
    pred = predictions[0]  # 最初のバッチを取得
    
    # グリッドごとの予測を取得
    grid_preds = pred.reshape(
        (grid_size, grid_size, num_boxes, 5 + num_classes)
    )
    
    # 位置情報を取得
    cell_x = grid_preds[..., 0]  # セル相対x
    cell_y = grid_preds[..., 1]  # セル相対y
    cell_w = grid_preds[..., 2]  # セル相対w
    cell_h = grid_preds[..., 3]  # セル相対h
    confidence = grid_preds[..., 4]  # 信頼度
    class_probs = grid_preds[..., 5:]  # クラス確率
    
    # 絶対座標に変換
    absolute_x = (cell_x + np.arange(grid_size)) / grid_size
    absolute_y = (cell_y + np.arange(grid_size)[:, np.newaxis]) / grid_size
    absolute_w = cell_w * (1 / grid_size)  # 相対幅
    absolute_h = cell_h * (1 / grid_size)  # 相対高さ
    
    # 最も確信度の高いクラスを取得
    class_ids = np.argmax(class_probs, axis=-1)
    class_confidences = np.max(class_probs, axis=-1)
    
    # 全体の信頼度を計算
    overall_confidence = confidence * class_confidences
    
    # 検出結果をリストに格納
    detections = []
    
    for i in range(grid_size):
        for j in range(grid_size):
            for b in range(num_boxes):
                conf = overall_confidence[i, j, b]
                if conf > 0.3:  # 信頼度しきい値
                    detection = {
                        'x': absolute_x[i, j, b],
                        'y': absolute_y[i, j, b],
                        'w': absolute_w[i, j, b],
                        'h': absolute_h[i, j, b],
                        'confidence': conf,
                        'class_id': class_ids[i, j, b],
                        'class_prob': class_probs[i, j, b, class_ids[i, j, b]]
                    }
                    detections.append(detection)
    
    return detections

def visualize_yolo_predictions(image, detections, grid_size=7):
    """YOLOの検出結果を可視化"""
    # 画像サイズ
    img_h, img_w = image.shape[:2]
    
    # 画像のコピーを作成
    vis_image = image.copy()
    
    # ランダムな色を生成
    colors = plt.cm.rainbow(np.linspace(0, 1, 20))
    
    # 検出を描画
    for det in detections[:10]:  # 最初の10個のみ表示
        # バウンディングボックスを計算
        x1 = int((det['x'] - det['w']/2) * img_w)
        y1 = int((det['y'] - det['h']/2) * img_h)
        x2 = int((det['x'] + det['w']/2) * img_w)
        y2 = int((det['y'] + det['h']/2) * img_h)
        
        # クラスの色を選択
        color = colors[det['class_id']]
        color = tuple(int(c * 255) for c in color[:3])
        
        # バウンディングボックスを描画
        cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 3)
        
        # ラベルを描画
        label = f"Class {det['class_id']}: {det['confidence']:.2f}"
        cv2.putText(vis_image, label, (x1, y1-10),
                   cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
    
    plt.figure(figsize=(12, 8))
    plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
    plt.title("YOLO検出結果")
    plt.axis('off')
    plt.show()

# ダミーデータでテスト
dummy_predictions = np.random.random((1, 7, 7, 2, 25))  # batch_size=1
detections = yolo_decode_predictions(dummy_predictions)

print(f"検出された物体の数: {len(detections)}")
for i, det in enumerate(detections[:3]):
    print(f"物体 {i+1}: クラス={det['class_id']}, 信頼度={det['confidence']:.3f}")

## Exercise 26.2: NMS（Non-Maximum Suppression）の実装

複数のバウンディングボックスが重なっている場合に、最も良いものを選択するNMSを実装してください。

In [ ]:
def non_max_suppression(detections, iou_threshold=0.5, confidence_threshold=0.3):
    """Non-Maximum Suppressionの実装"""
    # 信頼度でフィルタリング
    filtered = [det for det in detections if det['confidence'] > confidence_threshold]
    
    if not filtered:
        return []
    
    # クラスごとに処理
    final_detections = []
    
    for class_id in range(20):  # クラス数
        # 同じクラスの検出を抽出
        class_detections = [det for det in filtered if det['class_id'] == class_id]
        
        if not class_detections:
            continue
        
        # 信頼度でソート（降順）
        class_detections.sort(key=lambda x: x['confidence'], reverse=True)
        
        # NMSを実行
        while class_detections:
            # 最も信頼度の高い検出を保持
            best = class_detections.pop(0)
            final_detections.append(best)
            
            # 重なりの度合いを計算（IoU）
            boxes_to_remove = []
            for i, det in enumerate(class_detections):
                iou = calculate_iou(best, det)
                if iou > iou_threshold:
                    boxes_to_remove.append(i)
            
            # 重なっているボックスを削除
            for i in reversed(boxes_to_remove):
                class_detections.pop(i)
    
    return final_detections

def calculate_iou(box1, box2):
    """2つのバウンディングボックスのIoUを計算"""
    # 交差領域を計算
    x1 = max(box1['x'] - box1['w']/2, box2['x'] - box2['w']/2)
    y1 = max(box1['y'] - box1['h']/2, box2['y'] - box2['h']/2)
    x2 = min(box1['x'] + box1['w']/2, box2['x'] + box2['w']/2)
    y2 = min(box1['y'] + box1['h']/2, box2['y'] + box2['h']/2)
    
    # 交差領域がなければIoU=0
    if x2 < x1 or y2 < y1:
        return 0.0
    
    # 交差面積を計算
    intersection = (x2 - x1) * (y2 - y1)
    
    # 各ボックスの面積
    area1 = box1['w'] * box1['h']
    area2 = box2['w'] * box2['h']
    
    # IoUを計算
    union = area1 + area2 - intersection
    iou = intersection / union if union > 0 else 0.0
    
    return iou

def visualize_nms_demo():
    """NMSのデモンストレーション"""
    # ダミーの検出結果
    detections = [
        {'x': 0.5, 'y': 0.5, 'w': 0.3, 'h': 0.4, 'confidence': 0.9, 'class_id': 0},
        {'x': 0.52, 'y': 0.48, 'w': 0.3, 'h': 0.4, 'confidence': 0.85, 'class_id': 0},
        {'x': 0.7, 'y': 0.6, 'w': 0.2, 'h': 0.2, 'confidence': 0.8, 'class_id': 0},
        {'x': 0.8, 'y': 0.7, 'w': 0.15, 'h': 0.15, 'confidence': 0.75, 'class_id': 0},
    ]
    
    # NMSを実行
    nms_detections = non_max_suppression(detections, iou_threshold=0.5)
    
    # 可視化
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    # NMS前
    ax = axes[0]
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title('NMS前')
    
    for i, det in enumerate(detections):
        x1 = det['x'] - det['w']/2
        y1 = det['y'] - det['h']/2
        rect = plt.Rectangle((x1, y1), det['w'], det['h'],
                            fill=False, edgecolor='red', linewidth=2,
                            label=f'{det["confidence"]:.2f}')
        ax.add_patch(rect)
        ax.text(det['x'], det['y'], f'{det["confidence"]:.2f}',
                ha='center', va='center', fontsize=10, color='red')
    
    # NMS後
    ax = axes[1]
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_title(f'NMS後 (IoU>{0.5})')
    
    for i, det in enumerate(nms_detections):
        x1 = det['x'] - det['w']/2
        y1 = det['y'] - det['h']/2
        rect = plt.Rectangle((x1, y1), det['w'], det['h'],
                            fill=False, edgecolor='blue', linewidth=3)
        ax.add_patch(rect)
        ax.text(det['x'], det['y'], f'{det["confidence"]:.2f}',
                ha='center', va='center', fontsize=10, color='blue')
    
    plt.tight_layout()
    plt.show()
    
    print(f"NMS前の検出数: {len(detections)}")
    print(f"NMS後の検出数: {len(nms_detections)}")
    print(f"削除された検出数: {len(detections) - len(nms_detections)}")

# NMSデモの実行
visualize_nms_demo()

## Exercise 26.3: 実践的な物体検出

実際の画像を使って物体検出を実行してみましょう。

In [ ]:
def create_sample_image_with_objects(num_objects=3):
    """サンプル画像の作成"""
    # 空の画像を作成
    image = np.ones((448, 448, 3), dtype=np.uint8) * 200  # 灰色の背景
    
    # ランダムな色で円形のオブジェクトを作成
    colors = [
        (255, 0, 0),    # 赤
        (0, 255, 0),    # 緑
        (0, 0, 255),    # 青
        (255, 255, 0),  # 黄色
        (255, 0, 255),  # マゼンタ
    ]
    
    objects = []
    
    for i in range(num_objects):
        # ランダムな位置とサイズ
        x = np.random.randint(50, 400)
        y = np.random.randint(50, 400)
        radius = np.random.randint(30, 80)
        color = colors[i % len(colors)]
        
        # 円を描画
        cv2.circle(image, (x, y), radius, color, -1)
        
        # オブジェクト情報を保存
        objects.append({
            'x': x / 448,  # 正規化座標
            'y': y / 448,
            'w': (radius * 2) / 448,
            'h': (radius * 2) / 448,
            'class_id': i % 3,  # 0, 1, 2のクラス
        })
    
    return image, objects

# サンプル画像の作成
sample_image, ground_truth = create_sample_image_with_objects(3)

# 画像の表示
plt.figure(figsize=(8, 6))
plt.imshow(cv2.cvtColor(sample_image, cv2.COLOR_BGR2RGB))
plt.title("サンプル画像")
plt.axis('off')
plt.show()

print(f"地面の真理値（オブジェクト）:")
for i, obj in enumerate(ground_truth):
    print(f"  オブジェクト{i+1}: クラス={obj['class_id']}, 位置=({obj['x']:.3f}, {obj['y']:.3f}), サイズ=({obj['w']:.3f}, {obj['h']:.3f})")

In [ ]:
def evaluate_detections(ground_truth, predictions, iou_threshold=0.5):
    """検出結果の評価"""
    true_positives = 0
    false_positives = 0
    false_negatives = 0
    
    # グラウンドトゥルースに対応する予測を探す
    matched_gt = []
    matched_pred = []
    
    for gt in ground_truth:
        best_iou = 0
        best_pred_idx = -1
        
        for i, pred in enumerate(predictions):
            if i in matched_pred:
                continue  # すでにマッチ済み
            
            # 同じクラスのみ比較
            if gt['class_id'] != pred['class_id']:
                continue
            
            # IoUを計算
            iou = calculate_iou(gt, pred)
            
            if iou > best_iou:
                best_iou = iou
                best_pred_idx = i
        
        if best_iou > iou_threshold:
            true_positives += 1
            matched_pred.append(best_pred_idx)
            matched_gt.append(True)
        else:
            false_negatives += 1
            matched_gt.append(False)
    
    # マッチしなかった予測を偽陽性とする
    false_positives = len(predictions) - len(matched_pred)
    
    # 評価指標の計算
    precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) > 0 else 0
    recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) > 0 else 0
    f1 = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    
    return {
        'true_positives': true_positives,
        'false_positives': false_positives,
        'false_negatives': false_negatives,
        'precision': precision,
        'recall': recall,
        'f1': f1
    }

# 検出結果をシミュレーション
simulated_predictions = [
    {'x': 0.25, 'y': 0.25, 'w': 0.15, 'h': 0.15, 'confidence': 0.9, 'class_id': 0},
    {'x': 0.65, 'y': 0.7, 'w': 0.2, 'h': 0.2, 'confidence': 0.8, 'class_id': 1},
    {'x': 0.45, 'y': 0.45, 'w': 0.1, 'h': 0.1, 'confidence': 0.7, 'class_id': 2},
    {'x': 0.8, 'y': 0.8, 'w': 0.05, 'h': 0.05, 'confidence': 0.6, 'class_id': 0},  # 誤検出
]

# 評価
results = evaluate_detections(ground_truth, simulated_predictions)

print(f"検出結果の評価:")
print(f"  真陽性 (TP): {results['true_positives']}")
print(f"  偽陽性 (FP): {results['false_positives']}")
print(f"  偽陰性 (FN): {results['false_negatives']}")
print(f"  精度 (Precision): {results['precision']:.3f}")
print(f"  再現率 (Recall): {results['recall']:.3f}")
print(f"  F1スコア: {results['f1']:.3f}")

# 結果の可視化
plt.figure(figsize=(10, 6))

# 真陽性を緑で表示
for gt in ground_truth:
    x1 = (gt['x'] - gt['w']/2) * 448
    y1 = (gt['y'] - gt['h']/2) * 448
    rect = plt.Rectangle((x1, y1), gt['w']*448, gt['h']*448,
                        fill=False, edgecolor='green', linewidth=2,
                        label='Ground Truth')
    plt.gca().add_patch(rect)

# 予測を表示
for i, pred in enumerate(simulated_predictions):
    x1 = (pred['x'] - pred['w']/2) * 448
    y1 = (pred['y'] - pred['h']/2) * 448
    
    if i < 3:  # 真陽性
        color = 'blue'
        alpha = 1.0
        label = 'True Positive' if i == 0 else None
    else:  # 偽陽性
        color = 'red'
        alpha = 0.7
        label = 'False Positive'
    
    rect = plt.Rectangle((x1, y1), pred['w']*448, pred['h']*448,
                        fill=False, edgecolor=color, linewidth=2, alpha=alpha,
                        label=label)
    plt.gca().add_patch(rect)
    plt.text(pred['x']*448, pred['y']*448, f"{pred['confidence']:.2f}",
             ha='center', va='center', fontsize=10,
             color='blue' if i < 3 else 'red')

plt.xlim(0, 448)
plt.ylim(0, 448)
plt.gca().invert_yaxis()
plt.xlabel('X')
plt.ylabel('Y')
plt.title('物体検出の評価')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axis('equal')
plt.show()

## Challenge Problem: 物体検出システムの統合

以下の要件を満たす完全な物体検出システムを実装してください：

1. YOLOの特徴抽出部分
2. バウンディングボックスの予測
3. NMSによる後処理
4. 評価指標の計算
5. 結果の可視化

In [ ]:
class SimpleObjectDetector:
    """シンプルな物体検出システム"""
    def __init__(self, grid_size=7, num_boxes=2, num_classes=3):
        self.grid_size = grid_size
        self.num_boxes = num_boxes
        self.num_classes = num_classes
        self.model = None
    
    def build_model(self):
        """検出モデルを構築"""
        inputs = keras.Input(shape=(448, 448, 3))
        
        # 簡単な特徴抽出器
        x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
        x = layers.MaxPooling2D(2)(x)
        x = layers.Conv2D(256, 3, padding='same', activation='relu')(x)
        x = layers.GlobalAveragePooling2D()(x)
        
        # 全結合層
        x = layers.Dense(1024, activation='relu')(x)
        x = layers.Dropout(0.5)(x)
        
        # YOLO出力層
        output = layers.Dense(
            self.grid_size * self.grid_size * (self.num_boxes * 5 + self.num_classes)
        )(x)
        
        self.model = keras.Model(inputs=inputs, outputs=output)
        return self.model
    
    def predict(self, image):
        """画像からの物体検出"""
        if self.model is None:
            self.build_model()
        
        # 画像を前処理
        if image.shape != (448, 448, 3):
            image = cv2.resize(image, (448, 448))
        
        # モデルで予測
        prediction = self.model.predict(image[np.newaxis, ...], verbose=0)[0]
        
        # 予測をデコード
        detections = yolo_decode_predictions(
            prediction[np.newaxis, ...],
            self.grid_size,
            self.num_boxes,
            self.num_classes
        )
        
        # NMSを適用
        detections = non_max_suppression(detections, iou_threshold=0.5)
        
        return detections
    
    def evaluate(self, ground_truth, predictions):
        """評価指標の計算"""
        return evaluate_detections(ground_truth, predictions)
    
    def visualize(self, image, detections, title="物体検出結果"):
        """検出結果の可視化"""
        # 画像のコピーを作成
        vis_image = image.copy()
        
        # ランダムな色を生成
        colors = plt.cm.rainbow(np.linspace(0, 1, self.num_classes))
        
        # 検出を描画
        for det in detections:
            # バウンディングボックスを計算
            x1 = int((det['x'] - det['w']/2) * 448)
            y1 = int((det['y'] - det['h']/2) * 448)
            x2 = int((det['x'] + det['w']/2) * 448)
            y2 = int((det['y'] + det['h']/2) * 448)
            
            # クラスの色を選択
            color = colors[det['class_id']]
            color = tuple(int(c * 255) for c in color[:3])
            
            # バウンディングボックスを描画
            cv2.rectangle(vis_image, (x1, y1), (x2, y2), color, 2)
            
            # ラベルを描画
            label = f"C{det['class_id']}: {det['confidence']:.2f}"
            cv2.putText(vis_image, label, (x1, y1-10),
                       cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 1)
        
        plt.figure(figsize=(10, 8))
        plt.imshow(cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB))
        plt.title(title)
        plt.axis('off')
        plt.show()

# 検出システムのテスト
detector = SimpleObjectDetector()

# 複数のテスト画像を作成
test_images = []
test_ground_truths = []

for i in range(3):
    img, gt = create_sample_image_with_objects(num_objects=np.random.randint(2, 5))
    test_images.append(img)
    test_ground_truths.append(gt)

# 予測（ダミーデータ）
all_predictions = []
all_ground_truths = []

for img, gt in zip(test_images, test_ground_truths):
    # ダミーの予測を生成
    predictions = []
    
    for obj in gt:
        # 少しノイズを加えた予測を生成
        pred = obj.copy()
        pred['x'] += np.random.normal(0, 0.05)
        pred['y'] += np.random.normal(0, 0.05)
        pred['w'] += np.random.normal(0, 0.05)
        pred['h'] += np.random.normal(0, 0.05)
        pred['confidence'] = np.random.uniform(0.5, 0.95)
        predictions.append(pred)
    
    # いくつかの偽陽性を追加
    if len(predictions) < 5:
        predictions.append({
            'x': np.random.uniform(0.2, 0.8),
            'y': np.random.uniform(0.2, 0.8),
            'w': np.random.uniform(0.1, 0.3),
            'h': np.random.uniform(0.1, 0.3),
            'confidence': np.random.uniform(0.3, 0.7),
            'class_id': np.random.randint(0, 3)
        })
    
    all_predictions.append(predictions)
    all_ground_truths.append(gt)

# 評価
for i, (img, gt, pred) in enumerate(zip(test_images, test_ground_truths, all_predictions)):
    print(f"\nテスト画像 {i+1}:")
    results = detector.evaluate(gt, pred)
    print(f"  精度: {results['precision']:.3f}, 再現率: {results['recall']:.3f}, F1: {results['f1']:.3f}")
    
    # 可視化
    detector.visualize(img, pred, f"テスト画像 {i+1}")

---

# Self-Check (理解度確認)

本日の学習内容を確認しましょう：

## 基礎知識
- [ ] 物体検出の基本的な概念（バウンディングボックス、クラス、信頼度）を理解した
- [ ] 2段階検出と一段階検出の違いを理解した
- [ ] YOLOの基本原理（グリッド分割、予測形式）を理解した
- [ ] NMS（Non-Maximum Suppression）の必要性を理解した

## 実践力
- [ ] YOLOの基本的な実装ができた
- [ ] バウンディングボックスの予測と変換を実装できた
- [ ] NMSを自前で実装できた
- [ ] 評価指標（精度、再現率、F1スコア）を理解し計算できた

## 深層理解
- [ ] 物体検出の課題（スケール変化、オクルージョンなど）を理解した
- [ ] 速度と精度のトレードオフを理解した
- [ ] なぜNMSが必要なのかを理解した
- [ ] 実際の応用での考慮事項を理解した

---

**お疲れ様でした！** Day 26はこれで終了です。

次回（Day 27）は「画像生成の基礎」を学び、GANやVAEなどの生成モデルについて学びます。

復習課題：
1. COCOデータセットを使って実際の物体検出モデルをトレーニングしてみる
2. さまざまなIoU閾値での性能を比較する
3. Real-time物体検出を実装し、Webカメラからリアルタイムに物体を検出する
4. 転移学習を使ってカスタムデータセットで物体検出モデルをファインチューニングする